In [ ]:
import ipywidgets.widgets as widgets
import sys
sys.path.append('/home/ballsbot/projects/ballsbot/python/lib')
from ballsbot.camera import get_images_and_cameras
from ballsbot.servos import get_controls
from ballsbot.manipulator import Manipulator

In [ ]:
image, camera = get_images_and_cameras()
display(image)

controller = widgets.Controller(index=0)  # http://html5gamepad.com
display(controller)

In [ ]:
car_controls = get_controls()

In [ ]:
manipulator = Manipulator(controller)